In [1]:
#pip install gym

In [2]:
import gym

env = gym.make("Taxi-v3",render_mode="ansi").env

env.reset(seed=0)

env = env.unwrapped

arr=env.render()
print(arr)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+




In [3]:
print("Action Space ", env.action_space)
print("State Space ", env.observation_space)

Action Space  Discrete(6)
State Space  Discrete(500)


In [4]:
# (taxi row, taxi column, passenger index, destination index)
state = env.encode(3, 1, 2, 0) 
print("State:", state)

env.s = state

arr=env.render()
print(arr)

State: 328
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+




In [5]:
env.P[env.s]

{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

In [6]:
# تحديد الحالة الابتدائية
state = env.encode(3, 1, 2, 0) 
# إسناد الحالة للبيئة
env.s = state

# عداد الحركات
epochs = 0
# عداد مرات  الجزاء
# لركوب أو إنزال خاطئ
penalties= 0

# حفظ الحالات للتحريك لاحقا
frames = [] # for animation
# متغير للدلالة على الوصول للهدف
done = False
# كرر طالما لم نصل للهدف
while not done:
    # اختيار الفعل بشكل عشوائي
    action = env.action_space.sample()
    # الانتقال للحالة التالية
    state, reward, done, _,_ = env.step(action)
    # جمع عدد مرات الجزاء
    if reward == -10:
        penalties += 1
          
    # إضافة إطار للعرض لاحقاً
    frames.append({
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1
      
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Timesteps taken: 871
Penalties incurred: 264


In [7]:
from IPython.display import clear_output
# مكتبة التوقيت
from time import sleep
# دالة لمحاكة التجوال
def print_frames(frames):
    actions=['North','South','East','West','Pick-up','Drop-of']
    for i, frame in enumerate(frames):
        # الحالة التالية
        env.s = frame['state']
        # مسح الإظهار
        clear_output(wait=True)
       # الإظهار
        arr=env.render()
        print(arr)
        # اسم الفعل
        # طباعة المعلومات
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {actions[frame['action']]}")
        print(f"Reward: {frame['reward']}")
        sleep(0.1) # sleep 0.1 second
# استدعاء الدالة        
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 871
State: 0
Action: Drop-of
Reward: 20


In [8]:
# تقييم الأداء دون التعلم

# عدد الحلقات
episodes = 100
# عداد الحركات
total_epochs = 0
# عداد العقوبات
total_penalties = 0

for _ in range(episodes):
    # حالة ابتدائية عشوائية
    state = env.reset()
    # عداد أخطاء الحلقة
    penalties=0
    # عداد أخطاء الحلقة
    reward = 0
    # عداد الحركات
    epochs=0
    done = False
    # الدوران حتى الوصول إلى الهدف
    while not done:
        
        # اختيار الفعل بشكل عشوائي
        action = env.action_space.sample()
        # الانتقال
        state, reward, done, _,_ = env.step(action)
        # عداد الأخطاء
        if reward == -10:
            penalties += 1
        # عداد الحركات
        epochs += 1
    # العدد الكلي للأخطاء
    total_penalties += penalties
    # العدد الكلي للحركات
    total_epochs += epochs
# طباعة التقييم
print(f"Results after {episodes} episodes:")
# وسطي عدد الحركات للوصول إلى الهدف
print(f"Average timesteps per episode: {(total_epochs / episodes)}")
# وسطي عدد الأخطاء
print(f"Average penalties per episode: {total_penalties / episodes}")

Results after 100 episodes:
Average timesteps per episode: 2635.75
Average penalties per episode: 855.8


In [9]:
# المكتبة نمباي
import numpy as np
# عدد الحالات
states=env.observation_space.n
# عدد الأفعال
actions=env.action_space.n
# إنشاء مصفوفة نمباي كلها أصفار
# عدد الصفوف هو عدد الحالات
# عدد الأعمدة هو عدد الأفعال
q_table = np.zeros([states, actions])
q_table

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [10]:
# تدريب الوكيل

# المعاملات الأساسية
alpha = 0.4
gamma = 0.6

# عدد الحلقات
episodes = 100000

for i in range(episodes):
    # إعادة التهيئة وتعيين حالة ابتدائية عشوائية
    state = env.reset()[0]
    # عداد الحركات
    epochs = 0
    # عداد الأخطاء في الركوب و التنزيل
    penalties = 0
    done = False
    # الدوران حتى الوصول للهدف
    while not done:
        action = np.argmax(q_table[state]) # القيمة العظمى لقيم التعلم
        # القيمة السابقة
        old_value = q_table[state, action]
        # الانتقال للحالة التالية
        next_state, reward, done, _,_ = env.step(action)
        next_state=next_state
        # أكبر قيمة في الحالة التالية
        next_max = np.max(q_table[next_state])
        # القيمة الجديدة
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        # تعديل القيمة
        q_table[state, action] = new_value
        # الانتقال للحالة التالية
        state = next_state
# الانتهاء من التعلم
print("Training finished.\n")

Training finished.



In [11]:
q_table

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [-2.36998839, -2.34639386, -2.33914219, -2.34639386, -2.27325184,
        -4.        ],
       [-2.04052738, -1.97946063, -1.95483418, -1.97946063, -0.7504    ,
        -4.        ],
       ...,
       [-1.45447014, -1.51029146, -1.45447014, -1.50226579, -4.        ,
        -4.        ],
       [-2.13270742, -2.12294538, -2.13270742, -2.18928092, -4.        ,
        -4.        ],
       [-0.736     , -0.736     , -0.736     ,  1.28      , -4.        ,
        -4.        ]])

In [12]:
# تقييم الأداء بعد التعلم

# عدد الحلقات
episodes = 100
# عداد الحركات
total_epochs = 0
# عداد العقوبات
total_penalties = 0

for _ in range(episodes):
    # حالة ابتدائية عشوائية
    state = env.reset()[0]
    # عداد أخطاء الحلقة
    penalties=0
    # عداد أخطاء الحلقة
    reward = 0
    epochs=0
    done = False
    # الدوران حتى الوصول إلى الهدف
    while not done:
        
        # اختيار الفعل الموافق لأكبر قيم تعلم للحالة الحالية
        action = np.argmax(q_table[state])
        # الانتقال
        state, reward, done, _,_ = env.step(action)
        # عداد الأخطاء
        if reward == -10:
            penalties += 1
        # عداد الحركات
        epochs += 1
    # العدد الكلي للأخطاء
    total_penalties += penalties
    # العدد الكلي للحركات
    total_epochs += epochs
# طباعة التقييم
print(f"Results after {episodes} episodes:")
# وسطي عدد الحركات للوصول إلى الهدف
print(f"Average timesteps per episode: {(total_epochs / episodes)}")
# وسطي عدد الأخطاء
print(f"Average penalties per episode: {total_penalties / episodes}")


Results after 100 episodes:
Average timesteps per episode: 13.16
Average penalties per episode: 0.0


In [13]:
# تحديد الحالة الابتدائية
state = env.encode(3, 1, 2, 0) 
print("State:", state)
# إسناد الحالة للبيئة
env.s = state
# الإظهار
arr=env.render()
print(arr)
# حفظ الحالات للتحريك لاحقا
frames = [] # for animation
# متغير للدلالة على الوصول للهدف
done = False
# كرر طالما لم نصل للهدف
while not done:
    # اختيار الفعل الموافق لأكبر قيم تعلم للحالة الحالية
    action = np.argmax(q_table[state])
    # الانتقال للحالة التالية
    state, reward, done, _, _ = env.step(action)
          
    # إضافة إطار للعرض لاحقاً
    frames.append({
        'state': state,
        'action': action,
        'reward': reward
        }
    )

# استدعاء التحريك
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 10
State: 0
Action: Drop-of
Reward: 20
